In [ ]:
import bw2data as bd
import bw2calc as bc
import stats_arrays as sa

In [ ]:
bd.projects.set_current("supply chain graph")

In [ ]:
if "steel bike" in bd.databases:
    del bd.databases["steel bike"]

In [ ]:
sb = bd.Database("steel bike")
sb.register()

In [ ]:
[node for node in sb]

In [ ]:
coal = sb.new_node(name="Coal", unit="kg", type=bd.labels.product_node_default)
coal_production = sb.new_node(name="Coal production", type=bd.labels.process_node_default)
steel = sb.new_node(name="Steel", unit="kg", type=bd.labels.product_node_default)
steel_production = sb.new_node(name="Steel production", type=bd.labels.process_node_default)
iron_mining = sb.new_node(name="Iron mining", type=bd.labels.process_node_default)
iron_ore = sb.new_node(name="Iron ore", unit="kg", type=bd.labels.product_node_default)
iron_ore_resource = sb.new_node(
    name="Iron ore", 
    unit="kg", 
    categories=("natural resource",), 
    type=bd.labels.biosphere_node_default
)
steel_bike_manufacture = sb.new_node(
    name="Steel bike manufacture", 
    type=bd.labels.process_node_default
)
steel_bike = sb.new_node(name="Steel bike", unit="unit", type=bd.labels.product_node_default)

In [ ]:
coal.save()
coal_production.save()
steel.save()
steel_production.save()
iron_mining.save()
iron_ore.save()
iron_ore_resource.save()
steel_bike_manufacture.save()
steel_bike.save()

In [ ]:
coal_production.new_edge(
    input=coal,
    amount=1,
    type=bd.labels.production_edge_default
).save()
steel_production.new_edge(
    input=steel,
    amount=1,
    type=bd.labels.production_edge_default
).save()
iron_mining.new_edge(
    input=iron_ore,
    amount=1,
    type=bd.labels.production_edge_default
).save()
steel_bike_manufacture.new_edge(
    input=steel_bike,
    amount=1,
    type=bd.labels.production_edge_default
).save()

In [ ]:
iron_mining.new_edge(
    input=iron_ore_resource,
    amount=1.25, 
    uncertainty_type=sa.TriangularUncertainty.id, 
    minimum=1,
    maximum=1.75, 
    type=bd.labels.biosphere_edge_default,
).save()

In [ ]:
lca = bc.LCA({iron_ore: 1})
lca.lci()

In [ ]:
lca.technosphere_matrix.todense()

In [ ]:
for k, v in lca.dicts.activity.reversed.items():
    print(k, bd.get_node(id=v))

for k, v in lca.dicts.product.reversed.items():
    print(k, bd.get_node(id=v))

In [ ]:
steel_production.new_edge(
    input=iron_ore,
    amount=1.1, 
    uncertainty_type=sa.TriangularUncertainty.id, 
    minimum=1,
    maximum=1.25, 
    type=bd.labels.consumption_edge_default,
).save()

In [ ]:
steel_production.new_edge(
    input=coal,
    amount=0.2, 
    uncertainty_type=sa.TriangularUncertainty.id, 
    minimum=0.1,
    maximum=0.5, 
    type=bd.labels.consumption_edge_default,
).save()

In [ ]:
steel_bike_manufacture.new_edge(
    input=steel,
    amount=8, 
    uncertainty_type=5, 
    minimum=4,
    maximum=15, 
    type=bd.labels.consumption_edge_default,
).save()

In [ ]:
iron_mining.new_edge(
    input=steel_bike,
    amount=0.001, 
    type=bd.labels.consumption_edge_default,
).save()
coal_production.new_edge(
    input=steel_bike,
    amount=0.001, 
    type=bd.labels.consumption_edge_default,
).save()
steel_production.new_edge(
    input=steel_bike,
    amount=0.001, 
    type=bd.labels.consumption_edge_default,
).save()

In [ ]:
bd.Method(("resources",)).write([
    (iron_ore_resource, {
        'amount': 3.141, 
        'uncertainty_type': sa.NormalUncertainty.id, 
        'loc': 3.141, 
        'scale': 0.5926535
    }),
])

In [ ]:
lca = bc.LCA({steel_bike: 1}, ("resources",))
lca.lci()
lca.lcia()

In [ ]:
lca.score